In [256]:
# %pip install prophet

In [257]:
import pandas as pd
from prophet import Prophet
from sklearn.preprocessing import OneHotEncoder
import pickle
import os
import numpy as np
from datetime import timedelta
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [258]:
# Carreguem les dades
df_tmp = pd.read_csv('data/preprocessed/dataMatrix/daily_covidMatrix.csv') 
df_est = pd.read_csv('data/preprocessed/dataMatrix/static_stateMatrix.csv')
df_clust = pd.read_csv('src_models/state_clusters.csv')

df = pd.merge(df_tmp, df_est, on='state', how='left')

In [259]:
df_est

,state,no_coverage,private_coverage,public_coverage,labor_cov_diff,bedsState_local_government,bedsNon_profit,bedsFor_profit,bedsTotal,population_state,...,pop_50-59,pop_60-69,pop_70-79,pop_80+,Low_SVI_CTGY,Moderate_Low_SVI_CTGY,Moderate_High_SVI_CTGY,High_SVI_CTGY,Metro,Non-metro
0,Alabama,10.0,67.3,36.5,5.1,1.4,0.8,0.9,3.1,4887871,...,0.133249,0.120537,0.075243,0.038527,8.99,15.97,43.99,31.04,76.84,23.16
1,Alaska,14.4,65.5,31.0,5.6,0.3,1.5,0.4,2.2,737438,...,0.127204,0.109072,0.050675,0.019748,0.79,83.41,7.29,8.51,67.41,32.59
2,Arizona,10.9,62.8,38.3,2.4,0.1,1.3,0.5,1.9,7171646,...,0.120016,0.113492,0.080306,0.040799,0.00,0.00,66.96,33.04,95.15,4.85
3,Arkansas,9.0,61.5,42.5,1.4,0.3,2.2,0.7,3.2,3013825,...,0.126968,0.115755,0.075878,0.040010,0.83,23.66,36.43,39.08,62.72,37.28
4,California,8.5,63.4,37.2,2.2,0.3,1.2,0.3,1.8,39557045,...,0.126974,0.104040,0.060900,0.035566,1.87,29.21,17.24,51.69,98.11,1.89
5,Colorado,8.1,70.5,32.0,1.3,0.4,1.1,0.4,1.9,5695564,...,0.124635,0.110858,0.060894,0.030431,35.54,44.99,15.25,4.23,89.18,10.82
6,Connecticut,5.6,71.8,34.4,1.5,0.1,2.0,0.0,2.0,3572665,...,0.148192,0.121102,0.072953,0.045950,13.84,37.17,48.99,0.00,94.94,5.06
7,Delaware,6.0,72.1,37.1,1.6,0.0,2.1,0.0,2.2,967171,...,0.137361,0.130649,0.084251,0.041448,0.00,81.43,18.57,0.00,100.00,0.00
8,District of Columbia,4.0,69.6,36.0,1.9,0.0,3.5,0.9,4.4,702455,...,0.018817,0.015807,0.009118,0.005588,0.00,0.00,100.00,0.00,100.00,0.00
9,Florida,13.5,61.9,36.9,4.1,0.4,1.3,0.9,2.6,21299325,...,0.134516,0.124694,0.091985,0.052912,4.46,16.99,55.28,23.27,96.65,3.35


In [260]:
# Select necessary data
df = df[["date", "state", "positiveIncrease", 'no_coverage', 'private_coverage',
       'public_coverage', 'labor_cov_diff', 'bedsTotal', 'population_state',
       'pop_density_state', 'pop_0-9', 'pop_10-19', 'pop_20-29', 'pop_30-39',
       'pop_40-49', 'pop_50-59', 'pop_60-69', 'pop_70-79', 'pop_80+',
       'Low_SVI_CTGY', 'Moderate_Low_SVI_CTGY', 'Moderate_High_SVI_CTGY',
       'High_SVI_CTGY', 'Metro']]

In [261]:
df = df[df["state"].isin(["Illinois", "Ohio"])]

df = df.rename(columns={'date': 'ds', 'positiveIncrease': 'y'})
df['ds'] = pd.to_datetime(df['ds'])  # Ensure datetime format

#Handle outliers clipping
df['y'] = df['y'].clip(lower=df['y'].quantile(0.01), upper=df['y'].quantile(0.99))

df_train = df[df['ds'] < '2021-03-01']
df_test = df[df['ds'] >= '2021-03-01']

In [262]:
model = Prophet(
    growth='linear',           # 'linear' or 'logistic'
    seasonality_mode='additive',  # 'additive' or 'multiplicative'
    yearly_seasonality=True,   # Auto-detected
    weekly_seasonality=True,
    daily_seasonality=True,   # Disable if not needed
    changepoint_prior_scale=0.15,  # Flexibility of trend (default: 0.05)
    holidays=None              # Add holiday dataframe if needed
)

model.add_regressor("pop_density_state")
# model_AL.add_regressor("pop_density_state")
model.add_regressor("Metro")
model.add_regressor("labor_cov_diff")
# model_AL.add_regressor("labor_cov_diff")
# model_AL.add_regressor("bedsTotal")
# model_AL.add_regressor("no_coverage")
# model_AL.add_regressor("pop_80+")

model.fit(df)

19:10:49 - cmdstanpy - INFO - Chain [1] start processing
19:10:49 - cmdstanpy - INFO - Chain [1] done processing


In [270]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

pop_density_state = {
    # 'Alabama': 96.509389,
    # 'Arizona': 63.135855048, 
    #'Arkansas': 57.919684465, 
    # 'California': 253.9065023, 
    # 'Florida': 397.01575445, 
    # 'Georgia': 182.26478911, 
    # 'Mississippi': 63.645625459,
    # 'District of Columbia': 11490.11954,
    # 'Hawaii': 221.17691552,
    # 'Texas': 109.86046804,
    'Illinois': 229.5111561,
    'Colorado': 54.955977559,
    #'Ohio': 286.07988455,

}
metro = {
    # 'Alabama': 76.84,
    # 'Arizona': 67.41, 
    #'Arkansas': 95.15,
    # 'District of Columbia': 100.00,
    # 'Hawaii': 100.00,
    # 'Texas': 00.00
    'Illinois': 88.61,
    #'Colorado': 89.18,
    'Ohio': 79.92,
}
labor_cov_diff = {
    # 'Alabama': 5.1,
    # 'Arizona': 2.4, 
    #'Arkansas': 1.4,
    # 'District of Columbia': 1.9,
    # 'Hawaii': 5.4,
    # 'Texas': 7.9,
    'Illinois': 3.7,
    #'Colorado': 1.3,
    'Ohio': 2.0,
}


for state, pop_dens in pop_density_state.items():
    df_test_st = df_test[df_test["state"] == state]

    future_st = model.make_future_dataframe(periods=len(df_test_st))
    future_st['pop_density_state'] = [pop_dens for _ in range(len(future_st))]
    future_st['Metro'] = [metro[state] for _ in range(len(future_st))]
    future_st['labor_cov_diff'] = [labor_cov_diff[state] for _ in range(len(future_st))]

    forecast_st = model.predict(future_st)
    mae_st = mean_absolute_error(df_test_st['y'], forecast_st.tail(len(df_test_st))['yhat'])
    print(state, mae_st)

Illinois 325.18640512097824


KeyError: 'Colorado'

In [267]:
df_test_st['y']

13367    1425.0
13368    1736.0
13369    2022.0
13370    1875.0
13371    1750.0
13372    1506.0
13373     735.0
Name: y, dtype: float64

In [268]:
forecast_st.tail(len(df_test_st))['yhat']

369    1310.964339
370    1637.125636
371    1689.996707
372    1975.734040
373    1954.931289
374    1829.830476
375    1218.376715
Name: yhat, dtype: float64